**transformation de yolo en mot**

In [ ]:
import os
import shutil
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split
import re

images_folder = r"C:\Users\rouat\Desktop\MMcows\Datasets\visual_data\visual_data\images\0725\cam_1"
labels_folder = r"C:\Users\rouat\Desktop\MMcows\Datasets\visual_data\visual_data\labels\combined\0725\cam_1"

output_root = os.path.join(os.path.expanduser("~"), "Desktop", "DS_dataset")

# Fonction pour trier par timestamp
def sort_by_timestamp(filename):
    match = re.match(r"(\d+)_", filename)
    return int(match.group(1)) if match else float('inf')

# Fonction de conversion YOLO → bbox
def yolo_to_bbox(yolo_line, img_w, img_h):
    parts = yolo_line.strip().split()
    object_id = int(parts[0])  
    x_center = float(parts[1]) * img_w
    y_center = float(parts[2]) * img_h
    width = float(parts[3]) * img_w
    height = float(parts[4]) * img_h
    bb_left = x_center - width / 2
    bb_top = y_center - height / 2
    return object_id, bb_left, bb_top, width, height

# Fonction principale pour préparer les séquences train/test
def prepare_sequence(images_list, split_name):
    sequence_path = os.path.join(output_root, split_name, "sequence_01")
    img1_folder = os.path.join(sequence_path, "img1")
    det_folder = os.path.join(sequence_path, "det")
    gt_folder = os.path.join(sequence_path, "gt")

    os.makedirs(img1_folder, exist_ok=True)
    os.makedirs(det_folder, exist_ok=True)
    os.makedirs(gt_folder, exist_ok=True)

    det_txt_path = os.path.join(det_folder, "det.txt")
    gt_txt_path = os.path.join(gt_folder, "gt.txt")  

    with open(det_txt_path, "w") as det_file, open(gt_txt_path, "w") as gt_file:
        for frame_id, image_name in enumerate(images_list, start=1):
            img_path = os.path.join(images_folder, image_name)
            label_name = os.path.splitext(image_name)[0] + ".txt"
            label_path = os.path.join(labels_folder, label_name)

            new_image_name = f"{frame_id:06d}.jpg"
            shutil.copy2(img_path, os.path.join(img1_folder, new_image_name))

            with Image.open(img_path) as img:
                img_w, img_h = img.size

            if not os.path.exists(label_path):
                continue

            with open(label_path, "r") as f:
                lines = f.readlines()

            for line in lines:
                object_id, x, y, w, h = yolo_to_bbox(line, img_w, img_h)

                class_id = 0

                det_line = f"{frame_id},-1,{x:.2f},{y:.2f},{w:.2f},{h:.2f},1.0,-1,-1,-1\n"
                det_file.write(det_line)

                gt_line = f"{frame_id},{object_id},{x:.2f},{y:.2f},{w:.2f},{h:.2f},1,0,1.0\n"
                gt_file.write(gt_line)

    print(f" Données générées pour '{split_name}' ➜ {sequence_path}")

image_files = sorted(os.listdir(images_folder), key=sort_by_timestamp)

train_images, test_images = train_test_split(
    image_files, test_size=0.2, random_state=42, shuffle=False
)

prepare_sequence(train_images, "train")
prepare_sequence(test_images, "test")


✅ Données générées pour 'train' ➜ C:\Users\rouat\Desktop\DS_dataset\train\sequence_01
✅ Données générées pour 'test' ➜ C:\Users\rouat\Desktop\DS_dataset\test\sequence_01


**evaluation**